In [2]:
import duckdb
import pandas as pd
import os

# This checks if the file actually exists before we try to load it
file_path = '../data/studies.txt'

if os.path.exists(file_path):
    print("Success: I found the 'studies.txt' file!")
else:
    print("Error: I cannot find the file. Check your folders.")

Success: I found the 'studies.txt' file!


In [3]:
# 1. Create a connection to DuckDB (it runs in memory)
con = duckdb.connect()

# 2. Write the SQL query
# We use 'read_csv_auto' which is a smart function that figures out the file format
query = """
SELECT *
FROM read_csv_auto('../data/studies.txt', delim='|', header=True, ALL_VARCHAR=True)
LIMIT 5
"""

# 3. Run the query and save it as a "DataFrame" (a table)
df_studies = con.execute(query).fetchdf()

# 4. Show the table
display(df_studies)

,nct_id,nlm_download_date_description,study_first_submitted_date,results_first_submitted_date,disposition_first_submitted_date,last_update_submitted_date,study_first_submitted_qc_date,study_first_posted_date,study_first_posted_date_type,results_first_submitted_qc_date,...,plan_to_share_ipd_description,created_at,updated_at,source_class,delayed_posting,expanded_access_nctid,expanded_access_status_for_nctid,fdaaa801_violation,baseline_type_units_analyzed,patient_registry
0,NCT05648474,None,2022-12-05,None,None,2024-10-22,2022-12-05,2022-12-13,ACTUAL,None,...,None,2025-11-19 04:18:22.808443,2025-11-19 04:18:22.808443,INDUSTRY,None,None,None,None,None,f
1,NCT06533592,None,2024-07-09,None,None,2024-10-21,2024-07-29,2024-08-01,ACTUAL,None,...,None,2025-11-19 04:18:22.808443,2025-11-19 04:18:22.808443,OTHER,None,None,None,None,None,None
2,NCT05799261,None,2022-12-23,None,None,2025-06-19,2023-03-22,2023-04-05,ACTUAL,None,...,None,2025-11-19 04:18:22.808443,2025-11-19 04:18:22.808443,OTHER,None,None,None,None,None,None
3,NCT04558593,None,2020-08-31,None,None,2025-06-17,2020-09-15,2020-09-22,ACTUAL,None,...,None,2025-11-19 04:18:22.808443,2025-11-19 04:18:22.808443,OTHER,None,None,None,None,None,f
4,NCT04531722,None,2020-08-13,None,None,2025-06-18,2020-08-24,2020-08-28,ACTUAL,None,...,None,2025-11-19 04:18:22.808443,2025-11-19 04:18:22.808443,OTHER,None,None,None,None,None,None


In [4]:
# Define the file paths for the other tables we need
studies_path = '../data/studies.txt'
sponsors_path = '../data/sponsors.txt'
conditions_path = '../data/conditions.txt'

# This is the "Master Query"
# We use LEFT JOIN to add info from other tables onto the main studies table
query = f"""
SELECT 
    S.nct_id, 
    S.study_type, 
    S.start_date, 
    S.completion_date,
    S.overall_status, 
    S.phase, 
    S.enrollment, 
    SP.name AS sponsor_name, 
    SP.agency_class AS sponsor_type,
    C.name AS condition_name
FROM read_csv_auto('{studies_path}', delim='|', header=True, ALL_VARCHAR=True) AS S
LEFT JOIN read_csv_auto('{sponsors_path}', delim='|', header=True, ALL_VARCHAR=True) AS SP 
    ON S.nct_id = SP.nct_id
LEFT JOIN read_csv_auto('{conditions_path}', delim='|', header=True, ALL_VARCHAR=True) AS C 
    ON S.nct_id = C.nct_id
WHERE 
    S.study_type = 'Interventional' 
    AND SP.lead_or_collaborator = 'lead'
LIMIT 1000
"""

# Run the query
print("Running Master Query... this might take 10-20 seconds...")
df_raw = con.execute(query).fetchdf()

# Check the shape (Rows, Columns)
print(f"Data Loaded! Shape: {df_raw.shape}")
display(df_raw.head())

Running Master Query... this might take 10-20 seconds...
Data Loaded! Shape: (0, 10)


,nct_id,study_type,start_date,completion_date,overall_status,phase,enrollment,sponsor_name,sponsor_type,condition_name


In [5]:
# Debugging: Check the exact spelling of values in the raw files

print("--- Unique Study Types in Studies Table ---")
query_debug_1 = f"""
SELECT DISTINCT study_type 
FROM read_csv_auto('{studies_path}', delim='|', header=True, ALL_VARCHAR=True)
LIMIT 10
"""
display(con.execute(query_debug_1).fetchdf())

print("\n--- Unique Sponsor Types in Sponsors Table ---")
query_debug_2 = f"""
SELECT DISTINCT lead_or_collaborator 
FROM read_csv_auto('{sponsors_path}', delim='|', header=True, ALL_VARCHAR=True)
LIMIT 10
"""
display(con.execute(query_debug_2).fetchdf())

--- Unique Study Types in Studies Table ---


,study_type
0,EXPANDED_ACCESS
1,None
2,INTERVENTIONAL
3,OBSERVATIONAL



--- Unique Sponsor Types in Sponsors Table ---


,lead_or_collaborator
0,lead
1,collaborator


In [7]:
# Revised Master Query with CORRECT SPELLING
query = f"""
SELECT 
    S.nct_id, 
    S.study_type, 
    S.start_date, 
    S.completion_date, 
    S.overall_status, 
    S.phase, 
    S.enrollment, 
    SP.name AS sponsor_name, 
    SP.agency_class AS sponsor_type,
    C.name AS condition_name
FROM read_csv_auto('{studies_path}', delim='|', header=True, ALL_VARCHAR=True) AS S
LEFT JOIN read_csv_auto('{sponsors_path}', delim='|', header=True, ALL_VARCHAR=True) AS SP 
    ON S.nct_id = SP.nct_id
LEFT JOIN read_csv_auto('{conditions_path}', delim='|', header=True, ALL_VARCHAR=True) AS C 
    ON S.nct_id = C.nct_id
WHERE 
    S.study_type = 'INTERVENTIONAL' 
    AND SP.lead_or_collaborator = 'lead'
LIMIT 1000
"""

print("Attempting to load data with corrected spelling...")
df_raw = con.execute(query).fetchdf()

print(f" Success! Data Loaded. Shape: {df_raw.shape}")
display(df_raw.head())

Attempting to load data with corrected spelling...
 Success! Data Loaded. Shape: (1000, 10)


,nct_id,study_type,start_date,completion_date,overall_status,phase,enrollment,sponsor_name,sponsor_type,condition_name
0,NCT05391750,INTERVENTIONAL,2022-10-19,2027-02-12,RECRUITING,PHASE1,72,Emory University,OTHER,Recurrent Plasma Cell Myeloma
1,NCT05391750,INTERVENTIONAL,2022-10-19,2027-02-12,RECRUITING,PHASE1,72,Emory University,OTHER,Refractory Plasma Cell Myeloma
2,NCT06462118,INTERVENTIONAL,2024-06-17,2024-10-22,COMPLETED,PHASE1,78,"Genentech, Inc.",INDUSTRY,Healthy Volunteers
3,NCT06312904,INTERVENTIONAL,2024-03-11,2024-08-14,COMPLETED,NA,100,"Chinese Academy of Medical Sciences, Fuwai Hos...",OTHER,Postoperative Pain
4,NCT06780813,INTERVENTIONAL,2025-01-10,2026-05-01,RECRUITING,NA,400,National and Kapodistrian University of Athens,OTHER,Supplemental Oxygen


In [9]:
# 1. Define the Full Query (No Limit)
full_query = f"""
SELECT 
    S.nct_id, 
    S.study_type, 
    S.start_date, 
    S.completion_date, 
    S.overall_status, 
    S.phase, 
    S.enrollment, 
    SP.name AS sponsor_name, 
    SP.agency_class AS sponsor_type,
    C.name AS condition_name
FROM read_csv_auto('{studies_path}', delim='|', header=True, ALL_VARCHAR=True) AS S
LEFT JOIN read_csv_auto('{sponsors_path}', delim='|', header=True, ALL_VARCHAR=True) AS SP 
    ON S.nct_id = SP.nct_id
LEFT JOIN read_csv_auto('{conditions_path}', delim='|', header=True, ALL_VARCHAR=True) AS C 
    ON S.nct_id = C.nct_id
WHERE 
    S.study_type = 'INTERVENTIONAL' 
    AND SP.lead_or_collaborator = 'lead'
"""

print("Loading FULL dataset... (This might take 1-2 minutes)")
df_all = con.execute(query).fetchdf()

print(f"Done! Total Studies Loaded: {df_all.shape[0]}")

# 2. Save to CSV so we can use it later easily
output_file = '../data/interventional_studies_raw.csv'
df_all.to_csv(output_file, index=False)
print(f"File saved to: {output_file}")

Loading FULL dataset... (This might take 1-2 minutes)
Done! Total Studies Loaded: 1000
File saved to: ../data/interventional_studies_raw.csv
